### 前準備

In [1]:
# ====================
# ライブラリのインストール
# ====================
! pip install --quiet torch==1.6.0
! pip install --quiet torchtext==0.7.0
! pip install --quiet pytorch-lightning==1.0.8
! pip install --quiet torchwordemb
! pip install --quiet optuna

     |████████████████████████████████| 748.8 MB 18 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
     |████████████████████████████████| 4.5 MB 9.8 MB/s 
     |████████████████████████████████| 1.2 MB 53.1 MB/s 
     |████████████████████████████████| 561 kB 9.8 MB/s 
     |████████████████████████████████| 829 kB 57.1 MB/s 
     |████████████████████████████████| 134 kB 70.2 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 308 kB 7.7 MB/s 
     |████████████████████████████████| 81 kB 11.7 MB/s 
     |███████████████████

### ここでランタイムを再起動

In [1]:
# ライブラリの読み込み
import os
import string
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader
# torchtextが自然言語処理やるようのPyTorchのライブラリ
from torchtext.data import Example, Field, Dataset, BucketIterator
import torchwordemb
from torchtext.vocab import FastText

# データセットのダウンロード
if os.path.isfile("/content/NewsAggregatorDataset.zip") == False:
    ! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
    ! unzip NewsAggregatorDataset.zip
    # 読込時のエラー回避のためダブルクォーテーションをシングルクォーテーションに置換
    ! sed -e 's/"/'\''/g' ./newsCorpora.csv > ./newsCorpora_re.csv

from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('/content/newsCorpora.csv', sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# df.locは単独および複数の要素の値を選択、取得、変更が可能
# isinはbool型を返す
df1 = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割 stratifyを設定することで訓練データとテストデータの指定した中身の割合を同じにすることができる
train, temp = train_test_split(df1, test_size=0.2, shuffle=True, random_state=0, stratify=df1['CATEGORY'])
test, valid = train_test_split(temp, test_size=0.5, shuffle=True, random_state=0, stratify=temp['CATEGORY'])

# データの保存
! mkdir -p /content/data/
train.to_csv('/content/data/train.txt', sep="\t", index=False)
test.to_csv('/content/data/test.txt', sep="\t", index=False)
valid.to_csv('/content/data/valid.txt', sep="\t", index=False)

--2022-03-10 06:59:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  44.5MB/s    in 0.6s    

2022-03-10 06:59:57 (44.5 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [2]:
# ====================
# 80. ID番号への変換
# ====================
# maketransの説明  maketrans(変換前文字列, 変換後文字列, 削除対象文字列)
# string.punctuationの中身  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))

# ラベルの辞書
label2id = {'b': 0, 't': 1, 'e':2, 'm':3}

# データの読み込み
# text用のFieldとlabel用のfieldを作る必要がある　テキストの方は単語分割を行うので上で作った単語分割用の関数を渡す
text_field = Field(sequential=True, use_vocab=True)
label_field = Field(sequential=False, use_vocab=False, is_target=True)
fields = [("x", text_field), ("t", label_field)]

# テキストの読み込み
def load_corpus(fname):
    examples = list()
    with open(fname, "r") as f:
        df  = pd.read_csv(fname, sep='\t')
        sentences = df["TITLE"]
        labels = df["CATEGORY"]
        for sentence, label in zip(sentences, labels):
            word_list = sentence.translate(table).split()
            label_id = label2id[label]
            # 単語のリストとラベルを渡せばTorchtextが望む形式に変換してくれる　Example.fromlist
            examples.append(Example.fromlist([word_list, label_id], fields))
        return Dataset(examples, fields)

dataset_train = load_corpus("/content/data/train.txt")
dataset_val = load_corpus("/content/data/valid.txt")
dataset_test = load_corpus("/content/data/test.txt")

# 語彙を登録（訓練データに含まれる単語にIDを割り振る） min_freq=2 2回以上出てきた単語すべてにIDを割り振る
text_field.build_vocab(dataset_train, min_freq=2)

# バッチサイズ
batch_size = 1

# データセットオブジェクトからデータローダーを作成　BucketIteratorはdataloaderに変換できる
dataloader_train = BucketIterator(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = BucketIterator(dataset_val, batch_size=batch_size, shuffle=False)
dataloader_test = BucketIterator(dataset_test, batch_size=batch_size, shuffle=False)

# 与えられた単語列に対して、ID番号列を返す関数
text = "I have a pen"
def return_id(sentence, text_field=text_field):
    return [ text_field.vocab.stoi[word] for word in sentence.translate(table).split()]
print(f'テキスト：{text}')
print(f'ID列：{return_id(text)}')

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


テキスト：I have a pen
ID列：[81, 227, 20, 0]


/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
# ============
# 81. RNNによる予測
# ============
class RNN(pl.LightningModule):

    # 埋め込み層(nn.Embeddingは単語IDを与えるとone-hotベクトルに変換した後, n_embedのサイズのベクトルに変換する), 隠れ層, 全結合層の定義 
    # n_inputは単語の種類 n_embedは単語ベクトルのサイズ, n_hiddenは文ベクトル, n_layersはlstmが何層あるか, bidirectionalはlstmを双方向にするか
    def __init__(self, n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional):
        super(RNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_embed, padding_idx=1)
        self.lstm = nn.LSTM(input_size=n_embed, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output)
    
    # 順伝播
    # oとhが同じものでoを出力として扱う それにfcをかけるとラベルになる
    def forward(self, x):
        o, (h, c) = self.lstm(self.embed(x))
        return self.fc(o[:, -1, :])

! rm -r model
! rm -r lightning_logs
# 単語の種類
n_input = len(text_field.vocab)
# 単語ベクトルの次元
n_embed = 300
n_hidden = 300
n_layers = 1
n_output = len(label2id)
dropout = 0.1
bidirectional = False

model = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional)

# 先頭10件の予測値取得
for i, data in enumerate(dataloader_train):
    if i>9:
        break
    result = torch.tensor([data.x[i].item() for i in range(len(data.x))])
    print(torch.softmax(model(result.unsqueeze(0)), dim=-1))

rm: cannot remove 'model': No such file or directory
rm: cannot remove 'lightning_logs': No such file or directory
tensor([[0.2443, 0.2448, 0.2550, 0.2559]], grad_fn=<SoftmaxBackward>)
tensor([[0.2687, 0.2516, 0.2441, 0.2356]], grad_fn=<SoftmaxBackward>)
tensor([[0.2184, 0.2714, 0.2560, 0.2542]], grad_fn=<SoftmaxBackward>)
tensor([[0.2706, 0.2529, 0.2331, 0.2435]], grad_fn=<SoftmaxBackward>)
tensor([[0.2577, 0.2626, 0.2457, 0.2341]], grad_fn=<SoftmaxBackward>)
tensor([[0.2246, 0.2626, 0.2575, 0.2554]], grad_fn=<SoftmaxBackward>)
tensor([[0.2890, 0.2573, 0.2176, 0.2360]], grad_fn=<SoftmaxBackward>)
tensor([[0.2345, 0.2697, 0.2568, 0.2390]], grad_fn=<SoftmaxBackward>)
tensor([[0.2491, 0.2566, 0.2420, 0.2523]], grad_fn=<SoftmaxBackward>)
tensor([[0.2655, 0.2603, 0.2391, 0.2352]], grad_fn=<SoftmaxBackward>)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
# ============
# 82. 確率的勾配降下法による学習
# ============
class RNN(pl.LightningModule):

    # 埋め込み層, 隠れ層, 全結合層の定義 
    # n_inputは単語の種類 n_embedは単語ベクトルのサイズ, n_hiddenは文ベクトル, n_layersはlstmが何層あるか, bidirectionalはlstmを双方向にするか
    def __init__(self, n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr):
        super(RNN, self).__init__()
        self.lr = lr
        # 埋め込み層　nn.Embeddingは単語IDを与えるとone-hotベクトルに変換した後, n_embedのサイズのベクトルに変換する
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_embed, padding_idx=1)
        # lstm層
        self.lstm = nn.LSTM(input_size=n_embed, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        # 全結合層
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output)
    
    # 順伝播
    # oとhが同じものでoを出力として扱う それにfcをかけるとラベルになる
    def forward(self, x):
        o, (h, c) = self.lstm(self.embed(x))
        return self.fc(o[-1])

    # 訓練用データのバッチを受け取って損失を計算
    def training_step(self, batch, batch_idx):
        x, t = batch
        # 予測したラベルがy
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("train_loss", loss)
        # pl.LightningModuleはlossだけ返せばbackwardは勝手にやってくれる
        return loss
    
    # 検証用データのバッチを受け取って損失を計算
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("val_loss", loss)

    # 評価用データのバッチを受け取って分類の正解率を計算
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        y = torch.argmax(y, dim=1)

        accuracy = torch.sum(t == y).item() / (len(y) * 1.0)
        self.log("test_acc", accuracy)

    # 損失関数を設定
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    # 最適化手法を設定
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)

In [ ]:
# バッチサイズ
batch_size = 1

# データセットオブジェクトからデータローダーを作成　BucketIteratorはdataloaderに変換できる
dataloader_train = BucketIterator(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = BucketIterator(dataset_val, batch_size=batch_size, shuffle=False)
dataloader_test = BucketIterator(dataset_test, batch_size=batch_size, shuffle=False)

# 訓練
! rm -r model
! rm -r lightning_logs
# 単語の種類
n_input = len(text_field.vocab)
# 単語ベクトルの次元
n_embed = 300
n_hidden = 300
n_layers = 1
n_output = len(label2id)
dropout = 0.3
bidirectional = False
lr = 0.001

model = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

# 訓練中にモデルを保存するための設定
checkpoint = pl.callbacks.ModelCheckpoint(
    # 検証用データにおける損失が最も小さいモデルを保存する
    monitor="val_loss", mode="min", save_top_k=1,
    # モデルファイル（重みのみ）を "model" というディレクトリに保存する
    save_weights_only=True, dirpath="model/"
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", patience=5
)

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=20, callbacks=[checkpoint, early_stopping])
trainer.fit(model, dataloader_train, dataloader_val)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=dataloader_test)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


rm: cannot remove 'model': No such file or directory


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | embed | Embedding | 2.8 M 
1 | lstm  | LSTM      | 722 K 
2 | fc    | Linear    | 1.2 K 


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=11.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.4849, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.8175),
 'train_loss': tensor(0.0005, device='cuda:0'),
 'val_loss': tensor(0.5761, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.817


In [ ]:
# ============
# 83. ミニバッチ化・GPU上での学習
# ============
# バッチサイズ
batch_size = 32

# データセットオブジェクトからデータローダーを作成  BucketIteratorはdataloaderに変換できる
dataloader_train = BucketIterator(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = BucketIterator(dataset_val, batch_size=batch_size, shuffle=False)
dataloader_test = BucketIterator(dataset_test, batch_size=batch_size, shuffle=False)

# 訓練
! rm -r model
! rm -r lightning_logs
# 単語の種類
n_input = len(text_field.vocab)
# 単語ベクトルの次元
n_embed = 300
n_hidden = 300
n_layers = 1
n_output = len(label2id)
dropout = 0.3
lr = 0.1
bidirectional = False

model = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

# 訓練中にモデルを保存するための設定
checkpoint = pl.callbacks.ModelCheckpoint(
    # 検証用データにおける損失が最も小さいモデルを保存する
    monitor="val_loss", mode="min", save_top_k=1,
    # モデルファイル（重みのみ）を "model" というディレクトリに保存する
    save_weights_only=True, dirpath="model/"
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", patience=5
)

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=20, callbacks=[checkpoint, early_stopping])
trainer.fit(model, dataloader_train, dataloader_val)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=dataloader_test)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, use

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=8.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.8435, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.8326),
 'train_loss': tensor(0.0059, device='cuda:0'),
 'val_loss': tensor(1.3249, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.833


In [4]:
# ============
# 84. 単語ベクトルの導入
# ============
# 学習済み単語ベクトルの読み込み
text_field.build_vocab(dataset_train, vectors=FastText(language="en"), min_freq=2)

.vector_cache/wiki.en.vec: 6.60GB [02:36, 42.1MB/s]                            
100%|██████████| 2519370/2519370 [04:38<00:00, 9045.71it/s]


In [7]:
class RNN(pl.LightningModule):

    # 埋め込み層, 隠れ層, 全結合層の定義 
    # n_inputは単語の種類 n_embedは単語ベクトルのサイズ, n_hiddenは文ベクトル, n_layersはlstmが何層あるか, bidirectionalはlstmを双方向にするか
    def __init__(self, n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr):
        super(RNN, self).__init__()
        self.lr = lr
        # 埋め込み層　nn.Embeddingは単語IDを与えるとone-hotベクトルに変換した後, n_embedのサイズのベクトルに変換する
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_embed, padding_idx=1)
        # 単語ベクトルで重み付け
        self.embed.weight.data.copy_(text_field.vocab.vectors)
        # lstm層
        self.lstm = nn.LSTM(input_size=n_embed, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        # 全結合層
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output)
    
    # 順伝播
    # oとhが同じものでoを出力として扱う それにfcをかけるとラベルになる
    def forward(self, x):
        o, (h, c) = self.lstm(self.embed(x))
        return self.fc(o[-1])

    # 訓練用データのバッチを受け取って損失を計算
    def training_step(self, batch, batch_idx):
        x, t = batch
        # 予測したラベルがy
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("train_loss", loss)
        # pl.LightningModuleはlossだけ返せばbackwardは勝手にやってくれる
        return loss
    
    # 検証用データのバッチを受け取って損失を計算
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("val_loss", loss)

    # 評価用データのバッチを受け取って分類の正解率を計算
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        y = torch.argmax(y, dim=1)

        accuracy = torch.sum(t == y).item() / (len(y) * 1.0)
        self.log("test_acc", accuracy)

    # 損失関数を設定
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    # 最適化手法を設定
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)

In [10]:
# バッチサイズ
batch_size = 32

# データセットオブジェクトからデータローダーを作成  BucketIteratorはdataloaderに変換できる
dataloader_train = BucketIterator(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = BucketIterator(dataset_val, batch_size=batch_size, shuffle=False)
dataloader_test = BucketIterator(dataset_test, batch_size=batch_size, shuffle=False)

# 訓練
! rm -r model
! rm -r lightning_logs
# 単語の種類
n_input = len(text_field.vocab)
# 単語ベクトルの次元
n_embed = 300
n_hidden = 500
n_layers = 1
n_output = len(label2id)
dropout = 0.1
lr = 0.1
bidirectional = False

model = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

# 訓練中にモデルを保存するための設定
checkpoint = pl.callbacks.ModelCheckpoint(
    # 検証用データにおける損失が最も小さいモデルを保存する
    monitor="val_loss", mode="min", save_top_k=1,
    # モデルファイル（重みのみ）を "model" というディレクトリに保存する
    save_weights_only=True, dirpath="model/"
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", patience=5
)

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=20, callbacks=[checkpoint, early_stopping])
trainer.fit(model, dataloader_train, dataloader_val)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=dataloader_test)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, use

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=13.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.6090, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.8235),
 'train_loss': tensor(0.2971, device='cuda:0'),
 'val_loss': tensor(0.7077, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.824


In [11]:
# ============
# 85. 双方向RNN・多層化
# ============

# バッチサイズ
batch_size = 16

# データセットオブジェクトからデータローダーを作成  BucketIteratorはdataloaderに変換できる
dataloader_train = BucketIterator(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_val = BucketIterator(dataset_val, batch_size=batch_size, shuffle=False)
dataloader_test = BucketIterator(dataset_test, batch_size=batch_size, shuffle=False)

! rm -r model
! rm -r lightning_logs
# 単語の種類
n_input = len(text_field.vocab)
# 単語ベクトルの次元
n_embed = 300
n_hidden = 500
# 多層化
n_layers = 2
n_output = len(label2id)
dropout = 0.1
lr = 0.1
# 双方向をTrueに変更
bidirectional = True 

model = RNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

# 訓練中にモデルを保存するための設定
checkpoint = pl.callbacks.ModelCheckpoint(
    # 検証用データにおける損失が最も小さいモデルを保存する
    monitor="val_loss", mode="min", save_top_k=1,
    # モデルファイル（重みのみ）を "model" というディレクトリに保存する
    save_weights_only=True, dirpath="model/"
)

early_stopping = pl.callbacks.EarlyStopping(
    monitor="val_loss", mode="min", patience=5
)

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=20, callbacks=[checkpoint, early_stopping])
trainer.fit(model, dataloader_train, dataloader_val)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=dataloader_test)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | embed | Embedding | 2.8 M 
1 | lstm  | LSTM      | 9.2 M 
2 | fc    | Linear    | 4.0 K 


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=12.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.5502, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.8394),
 'train_loss': tensor(0.0105, device='cuda:0'),
 'val_loss': tensor(0.5914, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.839
